In [ ]:
!pip install keras-tuner --upgrade

In [3]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [ ]:
(X_train,y_train),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()


In [5]:
X_train, X_test = X_train.astype('float32') / 255.0, X_test.astype('float32') / 255.0


In [6]:
def model_builder (hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])
  
  return model
  


In [7]:
tuner = kt.Hyperband(model_builder,
                      objective = 'val_accuracy',
                      max_epochs = 10,
                      factor = 3,
                      directory = 'my_dir',
                      project_name = 'keras_tuner')



In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(X_train, y_train, epochs = 50, validation_split = 0.2, callbacks = [stop_early])

Trial 30 Complete [00h 02m 23s]
val_accuracy: 0.8615833520889282

Best val_accuracy So Far: 0.8891666531562805
Total elapsed time: 00h 23m 06s


In [10]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


print(f"""
The optimal number of units in the first densely-connectedlayer is {best_hps.get('units')} 
and the optimal learning rate : {best_hps.get('learning_rate')}.
""")
      


The optimal number of units in the first densely-connectedlayer is 384 
and the optimal learning rate : 0.001.



In [12]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs = 50, validation_split = 0.2)
val_accuracy = history.history['val_accuracy']
best_epoch = val_accuracy.index(max(val_accuracy)) + 1

Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4936 - accuracy: 0.8278 - val_loss: 0.4287 - val_accuracy: 0.8483
Epoch 2/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3705 - accuracy: 0.8645 - val_loss: 0.3974 - val_accuracy: 0.8560
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3321 - accuracy: 0.8778 - val_loss: 0.3606 - val_accuracy: 0.8673
Epoch 4/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3069 - accuracy: 0.8864 - val_loss: 0.3447 - val_accuracy: 0.8746
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2862 - accuracy: 0.8935 - val_loss: 0.3343 - val_accuracy: 0.8780
Epoch 6/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2723 - accuracy: 0.8991 - val_loss: 0.3250 - val_accuracy: 0.8832
Epoch 7/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2600 - accuracy: 0.9028 - val_loss: 0.3194 - val_accuracy:

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(X_train, y_train, epochs = best_epoch, validation_split = 0.2)


Epoch 1/22
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4943 - accuracy: 0.8234 - val_loss: 0.4208 - val_accuracy: 0.8432
Epoch 2/22
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3693 - accuracy: 0.8659 - val_loss: 0.3621 - val_accuracy: 0.8683
Epoch 3/22
1500/1500 [==============================] - 11s 7ms/step - loss: 0.3333 - accuracy: 0.8779 - val_loss: 0.3318 - val_accuracy: 0.8810
Epoch 4/22
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3064 - accuracy: 0.8866 - val_loss: 0.3287 - val_accuracy: 0.8812
Epoch 5/22
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2882 - accuracy: 0.8925 - val_loss: 0.3385 - val_accuracy: 0.8783
Epoch 6/22
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2720 - accuracy: 0.8993 - val_loss: 0.3266 - val_accuracy: 0.8834
Epoch 7/22
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2575 - accuracy: 0.9034 - val_loss: 0.3154 - val_accuracy

In [14]:
eval_result = hypermodel.evaluate(X_test, y_test)
print('[test_loss, test accuracy] : ', eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4120 - accuracy: 0.8851
[test_loss, test accuracy] :  [0.41196656227111816, 0.8851000070571899]
